Proyecto Final
==============

### Tratamiento de Datos
### Máster de Ing. de Telecomunicación

# Autores

Juan Manuel Espinosa Moral ([100406523@alumnos.uc3m.es](mailto:100406523@alumnos.uc3m.es))

José Manuel García Núñez ([100544621@alumnos.uc3m.es](mailto:100544621@alumnos.uc3m.es))

In [ ]:
# Google Drive integration

# Libraries to work with Google Drive and the file system
from google.colab import drive
import os, sys

# Drive is mounted
drive.mount("/content/drive")

# Current directory is shown
print(os.getcwd())

# We change to work directory
directory_path = "/content/drive/MyDrive/Colab Notebooks/proyecto_td/"  # Define directory_path here
if not os.path.exists(directory_path):
  os.makedirs(directory_path)
  print(f"Directory created: {directory_path}")

os.chdir(directory_path) # Now change to the directory

# 1. Análisis de Variables de Entrada
Carga del dataset: datos del archivo JSON.
Categorías: las más frecuentes.
Rating y visualizaciones.
Análisis de correlación: categorias y variables de salida.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el archivo JSON
df = pd.read_json("full_format_recipes.json")

# Explorar las categorías más frecuentes
category_counts = df['categories'].explode().value_counts()
print("Categorías más frecuentes:\n", category_counts.head())

# Visualizar las categorías más frecuentes
category_counts.head(10).plot(kind='bar', title="Frecuencia de las categorías")
plt.ylabel("Número de recetas")
plt.show()

# Analizar la relación entre categorías y ratings, filtrando por las más frecuentes
df_exploded = df.explode('categories')  # Expandir listas de categorías
df_exploded = df_exploded.reset_index(drop=True)  # Resetear el índice

# Filtrar por las 10 categorías más frecuentes
top_categories = category_counts.head(10).index.tolist()
df_filtered = df_exploded[df_exploded['categories'].isin(top_categories)]

plt.figure(figsize=(12, 6))
sns.boxplot(x='categories', y='rating', data=df_filtered)
plt.xticks(rotation=90)
plt.title("Distribución de ratings por categoría (Top 10)")
plt.show()

In [ ]:
# Assuming 'df' is your DataFrame
correlation = df[['fat', 'protein', 'calories', 'sodium', 'rating']].corr()

print(correlation)

sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el archivo JSON
df = pd.read_json("full_format_recipes.json")

# Explorar las categorías más frecuentes
category_counts = df['categories'].explode().value_counts()
print("Categorías más frecuentes:\n", category_counts.head())

# Visualizar las categorías más frecuentes
category_counts.head(10).plot(kind='bar', title="Frecuencia de las categorías")
plt.ylabel("Número de recetas")
plt.show()

# Analizar la relación entre categorías y ratings
df_exploded = df.explode('categories')  # Expandir listas de categorías
# Reset the index to avoid duplicate index values
df_exploded = df_exploded.reset_index(drop=True)
plt.figure(figsize=(12, 6))
sns.boxplot(x='categories', y='rating', data=df_exploded)
plt.xticks(rotation=90)
plt.title("Distribución de ratings por categoría")
plt.show()


In [ ]:
import matplotlib.pyplot as plt


plt.scatter(df['fat'], df['rating'])
plt.xscale('log')  # Apply logarithmic scale to x-axis
plt.xlabel('Fat (grams) - Log Scale')
plt.ylabel('Rating')
plt.title('Fat vs. Rating (Log Scale)')
plt.show()

In [ ]:
# Define the bins for calories
bins = [0, 200, 400, 600, 800, 1000, float('inf')]
labels = ['0-200', '201-400', '401-600', '601-800', '801-1000', '1001+']

# Create a new column with the calorie bins
df['calorie_bins'] = pd.cut(df['calories'], bins=bins, labels=labels)

average_ratings = df.groupby('calorie_bins')['rating'].mean()

import matplotlib.pyplot as plt

plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Calorie Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Calories')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
# Create a new column with the calorie bins
bins_fat = [0, 10, 20, 30, 40, 50, float('inf')]
labels_fat = ['0-10', '11-20', '21-30', '31-40', '41-50', '51+']

df['fat_bins'] = pd.cut(df['fat'], bins=bins_fat, labels=labels_fat)

average_ratings = df.groupby('fat_bins')['rating'].mean()

plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Fat Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Fat')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
# Create a new column with the calorie bins
bins_protein = [0, 10, 20, 30, 40, 50, float('inf')]
labels_protein = ['0-10', '11-20', '21-30', '31-40', '41-50', '51+']

df['protein_bins'] = pd.cut(df['protein'], bins=bins_protein, labels=labels_protein)

average_ratings = df.groupby('protein_bins')['rating'].mean()

plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Protein Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Protein')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
top_20_categories = df_exploded['categories'].value_counts().head(20).index.tolist()
df_filtered = df_exploded[df_exploded['categories'].isin(top_20_categories)]

variables = ['fat', 'protein', 'calories', 'sodium', 'rating']
for variable in variables:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='categories', y=variable, data=df_filtered)
    plt.xticks(rotation=90)
    plt.title(f'{variable.capitalize()} Distribution by Top 20 Categories')
    plt.tight_layout()
    plt.show()

In [ ]:
import re
import spacy

# Load the spaCy language model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text, use_spacy=False):
    """
    Preprocesar texto: limpiar, tokenizar, lematizar y eliminar stopwords.

    Args:
    - text (str | list | float): Texto a procesar o lista de textos.
    - use_spacy (bool): Usar SpaCy para el procesamiento en lugar de NLTK.

    Returns:
    - str: Texto preprocesado.
    """
    # Si el texto es NaN o no es string/list, convertirlo a cadena vacía
    if not isinstance(text, (str, list)):
        text = ''

    # Si el texto es una lista, unir sus elementos en una sola cadena
    if isinstance(text, list):
        text = ' '.join(text)

    # Convertir a minúsculas
    text = text.lower()

    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-z\s]', '', text)

    if use_spacy:
        # Usar SpaCy para tokenización y lematización
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]
    else:
        # Tokenizar usando NLTK
        tokens = nltk.word_tokenize(text)

        # Eliminar stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        # Lematizar
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Unir tokens de nuevo en un solo string
    processed_text = ' '.join(tokens)
    return processed_text

# Manejar valores NaN en la columna 'categories'
df['categories'] = df['categories'].fillna('')

# Aplicar el pipeline al conjunto de datos
df['processed_categories'] = df['categories'].apply(preprocess_text, use_spacy=True)

# Inspeccionar el resultado
print(df[['categories', 'processed_categories']].head())



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'df' contains your DataFrame with the 'processed_categories' column

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed text
tfidf_matrix = vectorizer.fit_transform(df['processed_categories'])

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names, index=df.index)

# Print or inspect the resulting TF-IDF matrix or DataFrame
print(tfidf_df)  # Or inspect tfidf_matrix

In [ ]:
!pip install nltk==3.8.1
!pip install gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import numpy as np
from gensim.models import Word2Vec


# Preparado de datos de texto
sentences = [row.split() for row in df['processed_categories']]

# Entrenamiento del modelo con las condiciones normales

model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

# 3. Obtén los vectores de palabras y promediados

def get_category_vector(category_words):
  """Obtiene el vector promedio para una categoría."""
  vectors = [model.wv[word] for word in category_words if word in model.wv]
  if vectors:
    return np.mean(vectors, axis=0)
  else:
    return np.zeros(model.vector_size)  # Vector de ceros si no hay palabras en el vocabulario

df['category_vector'] = df['processed_categories'].apply(lambda x: get_category_vector(x.split()))

# Muestra los primeros 5 elementos del vector para las primeras 5 filas
for index in range(5):
  print(f"Fila {index}: {df['category_vector'][index][:5]}")



Testing de los metodos de vectorización TF-IDF y vec2sec

In [ ]:
from wordcloud import WordCloud

# Crear un diccionario de palabras y sus pesos TF-IDF
word_weights = dict(zip(feature_names, tfidf_matrix.sum(axis=0).tolist()[0]))

# nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_weights)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


from sklearn.decomposition import PCA

# Reducir la dimensionalidad a 2D
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(df['category_vector'].to_list())

# Graficar
plt.figure(figsize=(10, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
plt.show()